In [1]:
# =========================================================
# VietDish — 20-class Food Classifier (Single + Batch + Lookup w/ Image)
# =========================================================
# !pip -q install gradio==4.* pillow numpy tensorflow pandas

import os, json, numpy as np, pandas as pd
from PIL import Image
import gradio as gr
import tensorflow as tf
from pathlib import Path

# ----------------- CONFIG (SỬA THEO BẠN) -----------------
MODEL_PATH  = "/content/vnmesesdishes.h5"   # <— đường dẫn model đã train (.h5/.keras)
CLASS_NAMES = [
    "baba_nau_chuoi_dau",
    "banh_tet",
    "banh_bao",
    "banh_beo",
    "banh_bo",
    "banh_bot_loc",
    "banh_can",
    "banh_canh",
    "banh_chung",
    "banh_cong",
    "banh_cuon",
    "banh_da_cua",
    "banh_da_lon",
    "banh_duc",
    "banh_gai",
    "banh_giay",
    "banh_gio",
    "banh_khot",
    "banh_la",
    "banh_mi",
]
APP_NAME    = "VietDish — 20 món Việt"
TAU_ABSTAIN = 0.50
MAX_FILES   = 5
# ----------------------------------------------------------

# 1) Load model & lấy input shape
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"❌ Không tìm thấy model tại: {MODEL_PATH}")

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
try:
    input_h, input_w, input_c = model.input_shape[1:4]
    input_h = input_h or 224
    input_w = input_w or 224
    input_c = input_c or 3
except Exception:
    input_h, input_w, input_c = 224, 224, 3

# 2) Thông tin món (mô tả / vùng / nguyên liệu chính)
DISH_INFO = {
    "baba_nau_chuoi_dau": {"mo_ta":"Ba ba nấu chuối đậu: thịt ba ba nấu cùng chuối xanh, đậu phụ, nghệ và tía tô.","vung":"Miền Bắc","nguyen_lieu":"ba ba, chuối xanh, đậu phụ, nghệ, tía tô, hành, mẻ"},
    "banh_tet": {"mo_ta":"Bánh tét dạng đòn, nếp gói lá chuối, nhân đậu xanh/thịt mỡ.","vung":"Nam Bộ (phổ biến toàn quốc dịp Tết)","nguyen_lieu":"nếp, đậu xanh, thịt ba rọi, lá chuối"},
    "banh_bao": {"mo_ta":"Bánh hấp vỏ bột mì mềm, nhân thịt trứng cút/nấm mộc nhĩ.","vung":"Phổ biến toàn quốc","nguyen_lieu":"bột mì, men, thịt heo, trứng cút, mộc nhĩ"},
    "banh_beo": {"mo_ta":"Bánh bột gạo hấp trong chén nhỏ, ăn kèm tôm cháy, mỡ hành, nước mắm.","vung":"Huế, miền Trung","nguyen_lieu":"bột gạo, tôm, mỡ hành, nước mắm"},
    "banh_bo": {"mo_ta":"Bánh bột gạo/men nở, ruột rỗ tổ ong, vị ngọt nhẹ.","vung":"Nam Bộ","nguyen_lieu":"bột gạo, dừa, đường, men"},
    "banh_bot_loc": {"mo_ta":"Bánh bột lọc trong suốt, nhân tôm thịt, gói lá chuối hoặc trần.","vung":"Huế, miền Trung","nguyen_lieu":"bột năng, tôm, thịt heo, lá chuối"},
    "banh_can": {"mo_ta":"Bánh bột gạo đổ khuôn đất, ăn kèm mắm nêm, mỡ hành, xoài chua.","vung":"Ninh Thuận – Bình Thuận","nguyen_lieu":"bột gạo, trứng (tuỳ), hành, mắm nêm"},
    "banh_canh": {"mo_ta":"Sợi bột (gạo/lọc) to, ăn với nước dùng thịt, cua hay giò heo.","vung":"Phổ biến toàn quốc","nguyen_lieu":"bột gạo/bột năng, nước lèo, thịt/cua, hành ngò"},
    "banh_chung": {"mo_ta":"Bánh vuông gói lá dong, nếp – đậu xanh – thịt mỡ, luộc lâu.","vung":"Miền Bắc","nguyen_lieu":"nếp, đậu xanh, thịt ba rọi, lá dong"},
    "banh_cong": {"mo_ta":"Bánh chiên giòn khuôn “cọc”, topping tôm, đậu xanh.","vung":"Sóc Trăng – Nam Bộ","nguyen_lieu":"bột gạo, đậu xanh, tôm, hành lá"},
    "banh_cuon": {"mo_ta":"Bánh tráng mỏng cuốn nhân thịt mộc nhĩ, chan nước mắm, chả quế.","vung":"Miền Bắc","nguyen_lieu":"bột gạo, thịt heo, mộc nhĩ, hành phi"},
    "banh_da_cua": {"mo_ta":"Bánh đa đỏ ăn với gạch/cua đồng, rau rút, chả lá lốt.","vung":"Hải Phòng","nguyen_lieu":"bánh đa đỏ, cua đồng, rau rút, chả"},
    "banh_da_lon": {"mo_ta":"Bánh hấp nhiều lớp xanh tím (lá dứa/đậu xanh), dẻo thơm.","vung":"Nam Bộ","nguyen_lieu":"bột năng, bột gạo, cốt dừa, lá dứa/đậu xanh"},
    "banh_duc": {"mo_ta":"Bánh bột gạo/ngô, ăn mặn (mộc nhĩ thịt băm) hay ngọt (nước đường).","vung":"Miền Bắc","nguyen_lieu":"bột gạo/bột năng, mộc nhĩ, thịt băm hoặc đường"},
    "banh_gai": {"mo_ta":"Bánh nếp đen lá gai, nhân đậu dừa, dẻo thơm đặc trưng.","vung":"Nam Định – Thanh Hoá","nguyen_lieu":"lá gai, gạo nếp, đậu xanh, dừa, đường"},
    "banh_giay": {"mo_ta":"Bánh nếp giã dẻo, ăn kẹp chả hoặc chấm đậu.","vung":"Miền Bắc","nguyen_lieu":"gạo nếp, muối, chả lụa"},
    "banh_gio": {"mo_ta":"Bánh hình chóp, bột gạo tẻ, nhân thịt mộc nhĩ, ăn nóng.","vung":"Miền Bắc","nguyen_lieu":"bột gạo, thịt heo, mộc nhĩ, hành khô"},
    "banh_khot": {"mo_ta":"Bánh đổ khuôn nhỏ, bột gạo + tôm, ăn kèm rau sống nước mắm.","vung":"Vũng Tàu – Nam Bộ","nguyen_lieu":"bột gạo, tôm, nghệ, nước mắm, rau sống"},
    "banh_la": {"mo_ta":"Nhóm bánh gói lá (chuối/dong) nhân ngọt/mặn tuỳ vùng.","vung":"Nhiều vùng","nguyen_lieu":"bột gạo/nếp, nhân đậu/cốt dừa hoặc mặn"},
    "banh_mi": {"mo_ta":"Ổ bánh mì vỏ giòn, kẹp pate–chả–thịt nướng–đồ chua.","vung":"Phổ biến toàn quốc","nguyen_lieu":"bột mì, pate/chả/thịt, dưa chua, rau mùi"},
}

# 2b) URL ảnh minh hoạ cho từng món — BẠN DÁN LINK Ở ĐÂY
DISH_IMAGE_URLS = {
    "baba_nau_chuoi_dau": "/content/baba.jpg",  # ví dụ: "https://.../baba.jpg"
    "banh_tet": "/content/banh tet.jpg",
    "banh_bao": "/content/bánh bao.webp",
    "banh_beo": "/content/banhbeo.jpg",
    "banh_bo": "/content/bánh bò.webp",
    "banh_bot_loc": "/content/bánh bột lộc.webp",
    "banh_can": "/content/bánh căn.png",
    "banh_canh": "/content/bánh canh.jpg",
    "banh_chung": "/content/bánh chưng.png",
    "banh_cong": "/content/bánh cống.webp",
    "banh_cuon": "/content/banhcuon.jpg",
    "banh_da_cua": "/content/bánh cống.webp",
    "banh_da_lon": "/content/da lợn.webp",
    "banh_duc": "/content/bánh đúc.jpg",
    "banh_gai": "/content/bánh gai.jpg",
    "banh_giay": "/content/bánh giày.webp",
    "banh_gio": "/content/bánh giò.jpg",
    "banh_khot": "/content/bánh khọt.webp",
    "banh_la": "/content/bánh lá.jpg",
    "banh_mi": "/content/banh-mi-thit-sai-gon.jpg",
}
# ----------------------------------------------------------

def _theme_vars(mode="Light"):
    return ""  # giữ đơn giản

BASE_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&family=Outfit:wght@700;800&display=swap');
.gradio-container { max-width: 1024px !important; }
* { letter-spacing:.1px; font-family: Inter, system-ui, -apple-system, Segoe UI; }
.header { padding: 14px 10px; border-radius: 12px; margin-bottom: 8px; }
.slab { background: rgba(0,0,0,.02); border:1px solid rgba(0,0,0,.08); border-radius:12px; padding:12px; }
.card { background: rgba(0,0,0,.03); border:1px solid rgba(0,0,0,.08); border-radius:12px; padding:12px; font-weight:600; }
.gr-image { border:1px solid rgba(0,0,0,.08); border-radius: 10px; }
"""

# ----------------- TIỀN XỬ LÝ ẢNH -----------------
def _to_rgb_or_gray(pil_img, want_c):
    return pil_img.convert("L") if want_c == 1 else pil_img.convert("RGB")

def preprocess(pil_img):
    img = _to_rgb_or_gray(pil_img, input_c)
    img = img.resize((input_w, input_h), Image.BILINEAR)
    arr = np.array(img, dtype=np.float32) / 255.0
    if input_c == 1:
        arr = arr[..., np.newaxis]
    arr = arr[np.newaxis, ...]      # (1,H,W,C)
    return arr

# ----------------- DỰ ĐOÁN -----------------
def _to_probs(vec):
    v = np.asarray(vec, dtype=np.float32).ravel()
    if (v < 0).any() or abs(v.sum() - 1.0) > 1e-3:
        e = np.exp(v - v.max())
        v = e / (e.sum() + 1e-8)
    return v

def _predict_core(pil_img, topk=3):
    arr = preprocess(pil_img)
    raw = model.predict(arr, verbose=0)[0]
    probs = _to_probs(raw)
    k = min(len(probs), len(CLASS_NAMES))
    probs = probs[:k] / (probs[:k].sum() + 1e-8)
    idx = int(np.argmax(probs))
    pred_class = CLASS_NAMES[idx]
    conf = float(probs[idx])
    top_idx = np.argsort(probs)[-topk:][::-1]
    top_list = [(CLASS_NAMES[i], float(probs[i])) for i in top_idx]
    return pred_class, conf, {CLASS_NAMES[i]: float(probs[i]) for i in range(k)}, top_list

def predict_single(image, theme_choice):
    if image is None:
        return ("### —", {}, "Hãy tải một ảnh món ăn!", "—", None, _theme_vars(theme_choice))

    pred_class, conf, conf_dict, _ = _predict_core(image, topk=3)
    info = DISH_INFO.get(pred_class, {"mo_ta":"Chưa có dữ liệu.","vung":"-","nguyen_lieu":"-"})
    if conf < TAU_ABSTAIN:
        title_md = "## Chưa chắc"
        summary  = (
            f"**Chưa chắc** — độ tin cậy ~ {conf*100:.1f}%.\n\n"
            f"• Gợi ý: dùng ảnh rõ, không che, nền đơn giản.\n"
            f"• Món nghi ngờ nhất: **{pred_class}**\n\n"
            f"**Thông tin tham khảo:**\n"
            f"- Mô tả: {info.get('mo_ta','N/A')}\n"
            f"- Vùng: {info.get('vung','N/A')}\n"
            f"- Nguyên liệu chính: {info.get('nguyen_lieu','N/A')}"
        )
    else:
        title_md = f"## {pred_class}"
        summary  = (
            f"**Món dự đoán:** {pred_class} — **độ tin cậy ~ {conf*100:.1f}%**\n\n"
            f"- **Mô tả:** {info.get('mo_ta','N/A')}\n"
            f"- **Vùng:** {info.get('vung','N/A')}\n"
            f"- **Nguyên liệu chính:** {info.get('nguyen_lieu','N/A')}"
        )

    return (title_md, conf_dict, summary, "—", image, _theme_vars(theme_choice))

def predict_batch(files, theme_choice):
    if not files:
        return "Chọn tối đa 5 ảnh.", pd.DataFrame(), [], _theme_vars(theme_choice)
    files = files[:MAX_FILES]
    rows, gallery_items = [], []
    for fp in files:
        try:
            img = Image.open(fp)
            pred_class, conf, _, top_list = _predict_core(img, topk=3)
            top3_txt = "; ".join([f"{c}: {p*100:.1f}%" for c,p in top_list])
            rows.append({"File": Path(fp).name, "Top-1": pred_class, "Độ tin cậy": f"{conf*100:.2f}%", "Top-3": top3_txt})
            caption = f"{pred_class} — {conf*100:.1f}%"
            gallery_items.append((fp, caption))
        except Exception as e:
            rows.append({"File": Path(str(fp)).name, "Top-1": "Lỗi", "Độ tin cậy": "-", "Top-3": f"Lỗi xử lý: {e}"})
    df = pd.DataFrame(rows, columns=["File","Top-1","Độ tin cậy","Top-3"])
    note = f"Đã xử lý {len(files)} ảnh (tối đa {MAX_FILES})."
    return note, df, gallery_items, _theme_vars(theme_choice)

# ----------------- TRA CỨU MÓN (Dropdown + Ảnh) -----------------
def lookup_dish(dish_key):
    """Trả về (Markdown mô tả, URL ảnh hoặc None)."""
    if not dish_key:
        return "Chọn một món trong danh sách để xem thông tin.", None
    info = DISH_INFO.get(dish_key, {"mo_ta":"Chưa có dữ liệu.","vung":"-","nguyen_lieu":"-"})
    txt = (
        f"### {dish_key}\n"
        f"- **Mô tả:** {info.get('mo_ta','N/A')}\n"
        f"- **Vùng:** {info.get('vung','N/A')}\n"
        f"- **Nguyên liệu chính:** {info.get('nguyen_lieu','N/A')}"
    )
    url = DISH_IMAGE_URLS.get(dish_key) or None
    return txt, url  # gr.Image có thể nhận URL trực tiếp

# ----------------- UI -----------------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue"),
               css=BASE_CSS,
               title=APP_NAME) as demo:

    theme_vars = gr.HTML(_theme_vars("Light"))

    gr.HTML(f"""
    <div class="header">
      <h1>🇻🇳 {APP_NAME}</h1>
      <p>Tải ảnh món ăn → dự đoán 1 trong 20 món Việt và hiển thị mô tả, vùng, nguyên liệu chính.<br/>
      Có <b>Batch nhiều ảnh</b> (≤5) và tab <b>Tra cứu</b> kèm <b>ảnh minh hoạ</b>.</p>
    </div>
    """)

    with gr.Tabs():
        # --- TAB 1: ẢNH ĐƠN ---
        with gr.Tab("🔎 Dự đoán ảnh đơn"):
            with gr.Row():
                with gr.Column(scale=1, min_width=360):
                    inp = gr.Image(type="pil", sources=["upload","webcam"],
                                   label="Upload / Webcam", image_mode="RGB", height=320)
                    theme_toggle = gr.Radio(choices=["Light","Dark"], value="Light",
                                            label="Theme", interactive=True)
                    btn = gr.Button("🚀 Dự đoán", variant="primary")
                with gr.Column(scale=1, elem_classes=["slab"]):
                    big_title   = gr.Markdown("### —")
                    lbl         = gr.Label(label="Xác suất dự đoán (probabilities)")
                    txt         = gr.Markdown(value="—")
                    spacer_box  = gr.Markdown(value="—", elem_classes=["card"])
            with gr.Row():
                with gr.Column(scale=1):
                    out_img = gr.Image(label="Preview", height=260)
            gr.HTML('© 2025 — Study demo • Not a nutrition/health product')

            btn.click(fn=predict_single,
                      inputs=[inp, theme_toggle],
                      outputs=[big_title, lbl, txt, spacer_box, out_img, theme_vars])
            theme_toggle.change(lambda m: _theme_vars(m), inputs=theme_toggle, outputs=theme_vars)

        # --- TAB 2: BATCH ---
        with gr.Tab("🗂️ Batch nhiều ảnh (≤5)"):
            with gr.Row():
                with gr.Column(scale=1, min_width=360):
                    files = gr.Files(label="Chọn tối đa 5 ảnh (jpg/png/webp)",
                                     file_count="multiple", type="filepath")
                    theme_toggle_b = gr.Radio(choices=["Light","Dark"], value="Light",
                                              label="Theme", interactive=True)
                    btn_b = gr.Button("🚀 Dự đoán Batch", variant="primary")
                with gr.Column(scale=2, elem_classes=["slab"]):
                    note_md  = gr.Markdown("—")
                    table    = gr.Dataframe(headers=["File","Top-1","Độ tin cậy","Top-3"],
                                            interactive=False, wrap=True)
                    gallery  = gr.Gallery(label="Kết quả (ảnh + caption)",
                                          columns=3, height=320)

            btn_b.click(predict_batch,
                        inputs=[files, theme_toggle_b],
                        outputs=[note_md, table, gallery, theme_vars])
            theme_toggle_b.change(lambda m: _theme_vars(m), inputs=theme_toggle_b, outputs=theme_vars)

        # --- TAB 3: TRA CỨU + ẢNH ---
        with gr.Tab("📚 Tra cứu món (Dropdown + Ảnh)"):
            with gr.Column():
                dd = gr.Dropdown(choices=CLASS_NAMES, value=None, label="Chọn món",
                                 filterable=True, allow_custom_value=False, multiselect=False)
                info_box = gr.Markdown("—")
                info_img = gr.Image(label="Ảnh minh hoạ", height=280)
                dd.change(lookup_dish, inputs=dd, outputs=[info_box, info_img])

demo.launch(share=True, show_error=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2f3e89cc794c24bd86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
